In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
from datetime import datetime

: 

In [ ]:
# Config
TOLERANCIA_SINCRONIZACAO_S = 0.5     # Tolerância máxima (em seg) para juntar sensores
FREQ_DOWNSAMPLE = '1s'          # '1min' = 1 ponto por minuto. Tente '10s' para 10 segundos.
N_CLUSTERS = 3          # Número de clusters desejados
PESSOA_INICIAL = 38      # Escolher a pessoa (exemplo)
WINDOW_SIZE = 7           # Número de pontos por janela (tamanho da janela). Usar este valor para window_size em funções

In [ ]:
def carregar_dados_pessoa(pessoa_id):
    # Caminhos dos arquivos
    accel_file = f'acelerometro/acelerometro_{pessoa_id}.csv'
    
    gyro_file = f'giroscopio/giroscopio_{pessoa_id}.csv'
    
    # Carregar acelerômetro (CSV com cabeçalho)
    df_accel = pd.read_csv(accel_file)
    df_accel['timestamp'] = pd.to_datetime(df_accel['timestamp'])
    
    # Carregar giroscópio (TXT sem cabeçalho)
    df_gyro = pd.read_csv(gyro_file)
    df_gyro['timestamp'] = pd.to_datetime(df_gyro['timestamp'])
    
    return df_accel, df_gyro

In [ ]:
df_accel, df_gyro = carregar_dados_pessoa(38)
print(df_gyro)

In [ ]:
def sincronizar_dados(df_accel, df_gyro, TOLERANCIA_SINCRONIZACAO_S=None):
    """
    Sincroniza dados de acelerômetro e giroscópio baseado em timestamps
    e cria janelas de tempo com múltiplos pontos

    Args:
        TOLERANCIA_SINCRONIZACAO_S: (opcional) taxa de amostragem em segundos. Usada para ajustar a tolerância do merge_asof.

    Returns:
        DataFrame: Dados sincronizados e agrupados
    """
    # Merge dos dados usando o timestamp mais próximo
    df_accel = df_accel.sort_values('timestamp').reset_index(drop=True)
    df_gyro = df_gyro.sort_values('timestamp').reset_index(drop=True)

    # Determinar tolerância baseada na taxa de amostragem (se fornecida) ou usar 0.1s por padrão
    if TOLERANCIA_SINCRONIZACAO_S is not None:
        tol = pd.Timedelta(seconds=TOLERANCIA_SINCRONIZACAO_S)
    else:
        tol = pd.Timedelta(seconds=0.1)

    # Combinar timestamps próximos
    df_combined = pd.merge_asof(
        df_accel, 
        df_gyro, 
        on='timestamp', 
        direction='nearest',
        tolerance=tol,
        suffixes=('', '_gyro')
    )
    
    # Renomear colunas do giroscópio para gx, gy, gz
    df_combined = df_combined.rename(columns={
        'x_gyro': 'gx',
        'y_gyro': 'gy',
        'z_gyro': 'gz'
    })
    
    df_combined = df_combined.dropna()
    
    return df_combined


In [ ]:
print(dados_sincronizados := sincronizar_dados(df_accel, df_gyro))

In [ ]:
# # 'dados_sincronizados' é o seu DataFrame da célula 6
# print(f"Pontos originais: {len(dados_sincronizados)}")

# # 1. Definir o timestamp como o índice
# df_indexado = dados_sincronizados.set_index('timestamp')

# # 2. Fazer o Downsample (Reamostragem)
# # '1S' = agrupar dados em janelas de 1 segundo
# # .mean() = calcular a média de todas as 7 leituras dentro daquele segundo
# df_downsampled = df_indexado.resample('1T').mean()

# # 3. Limpar linhas que possam ter ficado vazias (se houver buracos nos dados)
# df_downsampled = df_downsampled.dropna()

# # 4. (Opcional) Voltar o timestamp para uma coluna, se sua função 'criar_janelas'
# #    não esperar que ele seja o índice.
# df_downsampled = df_downsampled.reset_index()

# print(f"Pontos após downsample para 1Hz: {len(df_downsampled)}")

# # AGORA, você usaria 'df_downsampled' na sua função 'criar_features_janela'

In [ ]:
def criar_features_janela(df_downsampled, window_size):
    """
    Cria features agrupando múltiplos pontos consecutivos em uma janela
    
    Args:
        df: DataFrame com dados sincronizados
        window_size: Número de pontos por janela (cluster point)
    
    Returns:
        numpy array: Features para clustering (cada linha = 1 janela com múltiplos pontos)
    """
    features_list = []
    
    # Percorrer os dados em janelas
    for i in range(0, len(df_downsampled) - window_size + 1, window_size):
        window = df_downsampled.iloc[i:i+window_size]
        
        # Concatenar todos os valores da janela em um único vetor de features
        feature_vector = []
        for _, row in window.iterrows():
            # Adicionar acelerômetro (x, y, z)
            feature_vector.extend([row['x'], row['y'], row['z']])
            # Adicionar giroscópio (gx, gy, gz)
            feature_vector.extend([row['gx'], row['gy'], row['gz']])
        
        features_list.append(feature_vector)
    
    return np.array(features_list)

In [ ]:
features_lista = criar_features_janela(dados_sincronizados, WINDOW_SIZE)
tam = len(features_lista[0])
print(tam)

In [ ]:
def aplicar_kmeans(features, n_clusters):
    """
    Aplica K-means clustering nas features
    
    Args:
        features: Array de features
        n_clusters: Número de clusters
    
    Returns:
        tuple: (modelo KMeans treinado, labels dos clusters, features normalizadas)
    """
    # Normalizar features
    scaler = StandardScaler()
    features_normalized = scaler.fit_transform(features)
    
    # Aplicar K-means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(features_normalized)
    
    return kmeans, labels, features_normalized

In [ ]:
def plotar_resultados(labels, pessoa_id, window_size):
    """
    Plota os resultados do clustering
    
    Args:
        labels: Labels preditos pelo K-means
        pessoa_id: ID da pessoa
        window_size: Tamanho da janela usada
    """
    plt.figure(figsize=(14, 6))
    
    # Criar eixo X (índice do conjunto de pontos)
    x = np.arange(len(labels))
    
    # Identificar clusters únicos
    unique, counts = np.unique(labels, return_counts=True)
    n_clusters = len(unique)
    
    # Paleta de cores dinâmica
    color_palette = plt.cm.tab10(np.linspace(0, 1, n_clusters))
    color_map = {cluster_id: color_palette[i] for i, cluster_id in enumerate(unique)}
    
    # Plot 1: Labels ao longo do tempo
    plt.subplot(1, 2, 1)
    colors = [color_map[label] for label in labels]
    plt.scatter(x, labels, c=colors, alpha=0.6, s=50)
    plt.xlabel('Índice do Conjunto de Pontos', fontsize=12)
    plt.ylabel('Cluster (Label Predito)', fontsize=12)
    plt.title(f'K-means Clustering - Pessoa {pessoa_id}\n(Cada ponto = {window_size} leituras)', fontsize=14)
    plt.yticks(unique, [f'Cluster {i}' for i in unique])
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Distribuição dos clusters
    plt.subplot(1, 2, 2)
    cluster_labels = [f'Cluster {i}' for i in unique]
    plt.bar(cluster_labels, counts, color=color_palette, alpha=0.7)
    plt.xlabel('Cluster', fontsize=12)
    plt.ylabel('Número de Conjuntos de Pontos', fontsize=12)
    plt.title('Distribuição dos Clusters', fontsize=14)
    plt.grid(True, alpha=0.3, axis='y')
    
    # Adicionar valores nas barras
    for i, count in enumerate(counts):
        plt.text(i, count, str(count), ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f'clustering_pessoa_{pessoa_id}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n✓ Gráfico salvo como 'clustering_pessoa_{pessoa_id}.png'")


In [ ]:
def analisar_pessoa(pessoa_id, window_size, n_clusters):
    """
    Análise completa de clusterização para uma pessoa (COM DOWNSAMPLING)
    """
    print(f"\n{'='*60}")
    print(f"ANÁLISE - PESSOA {pessoa_id}")
    print(f"{'='*60}")
    
    # 1. Carregar dados
    print("\n[1/6] Carregando dados...")
    df_accel, df_gyro = carregar_dados_pessoa(pessoa_id)
    print(f"   ✓ Acelerômetro: {len(df_accel)} pontos")
    print(f"   ✓ Giroscópio: {len(df_gyro)} pontos")

    # 2. Sincronizar dados
    print("\n[2/6] Sincronizando dados (tolerância={TOLERANCIA_SINCRONIZACAO_S}s)...")
    df_combined = sincronizar_dados(df_accel, df_gyro, TOLERANCIA_SINCRONIZACAO_S)
    print(f"   ✓ Dados sincronizados: {len(df_combined)} pontos (taxa original)")

    # 3. NOVO: Realizar Downsampling
    print(f"\n[3/6] Realizando Downsampling (frequência={FREQ_DOWNSAMPLE})...")
    df_final = realizar_downsample(df_combined, freq=FREQ_DOWNSAMPLE)
    print(f"   ✓ Pontos após downsample: {len(df_final)} pontos")

    # 4. Criar features em janelas (AGORA USANDO O DADO DOWNSAMPLED)
    print(f"\n[4/6] Criando features (janelas de {window_size} pontos)...")
    # NOTA: Usamos 'df_final' aqui, não mais 'df_combined'
    features = criar_features_janela(df_final, window_size)
    n_features_per_window = window_size * 6
    print(f"   ✓ {len(features)} conjuntos de pontos criados")
    print(f"   ✓ Cada conjunto representa {window_size} * {FREQ_DOWNSAMPLE}")

    # 5. Aplicar K-means
    print(f"\n[5/6] Aplicando K-means (k={n_clusters})...")
    kmeans, labels, features_normalized = aplicar_kmeans(features, n_clusters)
    print(f"   ✓ Clustering concluído")
    print(f"   ✓ Inércia: {kmeans.inertia_:.2f}")

    # 6. Plotar resultados
    print("\n[6/6] Gerando visualização...")
    plotar_resultados(labels, pessoa_id, window_size)

    # Estatísticas
    print(f"\n{'='*60}")
    print("ESTATÍSTICAS")
    print(f"{'='*60}")
    unique, counts = np.unique(labels, return_counts=True)
    for cluster_id, count in zip(unique, counts):
        percentage = (count / len(labels)) * 100
        print(f"Cluster {cluster_id}: {count} conjuntos ({percentage:.1f}%)")

    return df_final, features, labels, kmeans

In [ ]:
# Validação rápida: verifica que a criação de features respeita WINDOW_SIZE
# Executar após definir/rodar analisar_pessoa para validar comportamento
try:
    # Se as variáveis ainda não existirem no escopo, pedimos para executar as células anteriores primeiro
    assert 'dados_sincronizados' in globals() or 'df_combined' in globals(), \
        "Execute as células de carregamento/sincronização antes desta validação."

    # Obter df usado: prefer df_combined criado pela última análise ou dados_sincronizados
    df_to_use = globals().get('df_combined', globals().get('dados_sincronizados'))

    # Gerar features de teste com WINDOW_SIZE
    test_features = criar_features_janela(df_to_use, WINDOW_SIZE)

    assert len(test_features) > 0, "Nenhuma janela criada — verifique WINDOW_SIZE e o tamanho do dataframe."
    assert len(test_features[0]) == WINDOW_SIZE * 6, \
        f"Cada feature deve ter WINDOW_SIZE*6 elementos, esperado {WINDOW_SIZE*6}, obteve {len(test_features[0])}."

    print(f"✓ Validação: {len(test_features)} janelas criadas; cada janela tem {len(test_features[0])} features ({WINDOW_SIZE} pontos × 6).")
except AssertionError as ae:
    print(f"✗ Validação falhou: {ae}")
except Exception as e:
    print(f"✗ Erro na validação: {e}")

In [ ]:
def analisar_todas_pessoas(window_size, n_clusters):
    """
    Análise de clusterização para todas as pessoas
    
    Args:
        window_size: Número de pontos por janela
        n_clusters: Número de clusters
    """
    # IDs das pessoas disponíveis
    pessoas = [11, 12, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 
               29, 30, 31, 32, 33, 34, 36, 37, 38]
    
    resultados = {}
    
    print("\n" + "="*60)
    print("ANÁLISE DE TODAS AS PESSOAS")
    print("="*60)
    
    for pessoa_id in pessoas:
        try:
            df_combined, features, labels, kmeans = analisar_pessoa(
                pessoa_id, window_size, n_clusters
            )
            resultados[pessoa_id] = {
                'data': df_combined,
                'features': features,
                'labels': labels,
                'kmeans': kmeans
            }
        except Exception as e:
            print(f"\n⚠ Erro ao processar pessoa {pessoa_id}: {e}")
            continue
    
    # Sumário geral
    print(f"\n\n{'='*60}")
    print("SUMÁRIO GERAL")
    print(f"{'='*60}")
    print(f"Total de pessoas analisadas: {len(resultados)}/{len(pessoas)}")
    
    return resultados

In [ ]:
if __name__ == "__main__":
    print("\n" + "="*60)
    print("CLUSTERIZAÇÃO K-MEANS - ACELERÔMETRO E GIROSCÓPIO")
    print("="*60)
    print(f"Taxa de amostragem: {TOLERANCIA_SINCRONIZACAO_S}s")
    print(f"Número de clusters: {N_CLUSTERS}")
    print(f"Tamanho da janela: {WINDOW_SIZE} pontos (leituras por janela)")
    
    # Opção 1: Analisar apenas uma pessoa (recomendado para começar)
    print("\n" + "-"*60)
    print("Opção escolhida: Analisar uma pessoa primeiro")
    print("-"*60)
    
    df_combined, features, labels, kmeans = analisar_pessoa(
        PESSOA_INICIAL, 
        window_size=WINDOW_SIZE, 
        n_clusters=N_CLUSTERS
    )
    
    # Descomente a linha abaixo para analisar todas as pessoas
    # resultados = analisar_todas_pessoas(window_size=N_JANELAS, n_clusters=N_CLUSTERS)
    
    print("\n" + "="*60)
    print("ANÁLISE CONCLUÍDA!")
    print("="*60)
    print("\nPara analisar todas as pessoas, descomente a última linha do script.")